## TAREA 2

In [3]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=30b27e6dda37d48b568c66ef848fc82fb167e1971c8a80230d83bd785f41bca5
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [4]:
import pyspark
print(pyspark.__version__)

3.5.3


In [5]:
pip install findspark

In [6]:
import subprocess
java_home = subprocess.check_output('echo %JAVA_HOME%', shell=True).decode().strip()
spark_home = subprocess.check_output('echo %SPARK_HOME%', shell=True).decode().strip()
print('JAVA_HOME:', java_home)
print('SPARK_HOME:', spark_home)

JAVA_HOME: %JAVA_HOME%
SPARK_HOME: %SPARK_HOME%


In [38]:
import os
import sys
import findspark
findspark.init()
findspark.find()
from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F
import time

In [8]:
from pyspark import SparkContext

In [9]:
import findspark
findspark.init()

In [10]:
import requests

api_key = "73e3e0580fb16a75fdf023438b452211"
city = "Guerrero"
url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"

response = requests.get(url)
data = response.json()

print(data)


{'coord': {'lon': -100, 'lat': 17.6667}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'base': 'stations', 'main': {'temp': 17.72, 'feels_like': 17.95, 'temp_min': 17.72, 'temp_max': 17.72, 'pressure': 1020, 'humidity': 92, 'sea_level': 1020, 'grnd_level': 827}, 'visibility': 10000, 'wind': {'speed': 1.9, 'deg': 181, 'gust': 1.77}, 'rain': {'1h': 0.19}, 'clouds': {'all': 46}, 'dt': 1728873935, 'sys': {'country': 'MX', 'sunrise': 1728822763, 'sunset': 1728865149}, 'timezone': -21600, 'id': 3527213, 'name': 'Guerrero', 'cod': 200}


In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("OpenWeatherData") \
    .getOrCreate()

In [12]:
weather_data = {
    'city': data['name'],
    'temperature': data['main']['temp'],
    'description': data['weather'][0]['description'],
    'humidity': data['main']['humidity']
}

data_list = [weather_data]

rdd = spark.sparkContext.parallelize(data_list)

print(rdd.collect())


[{'city': 'Guerrero', 'temperature': 17.72, 'description': 'light rain', 'humidity': 92}]


In [13]:
def C_to_F (celsius):
    fahr = celsius * (9/5) + 32
    return celsius, fahr

In [14]:
temp_celsius = data ["main"]["temp"]
temp_celsius, temp_fahr = C_to_F(temp_celsius)
feels_like_celsius = data ["main"]["feels_like"]
feels_like_celsius, feels_like_fahr = C_to_F(feels_like_celsius)

In [15]:
temp_celsius

17.72

In [16]:
temp_fahr

63.896

Para considerar varias ciudades:

In [21]:
cities = [
    "Aguascalientes, Mexico",
    "Mexicali, Mexico",
    "La Paz, Mexico",
    "Campeche, Mexico",
    "Tuxtla Gutiérrez, Mexico",
    "Chihuahua, Mexico",
    "Saltillo, Mexico",
    "Colima, Mexico",
    "Durango, Mexico",
    "Guanajuato, Mexico",
    "Chilpancingo, Mexico",
    "Pachuca, Mexico",
    "Guadalajara, Mexico",
    "Toluca, Mexico",
    "Morelia, Mexico",
    "Cuernavaca, Mexico",
    "Tepic, Mexico",
    "Monterrey, Mexico",
    "Oaxaca de Juárez, Mexico",
    "Puebla, Mexico",
    "Querétaro, Mexico",
    "Chetumal, Mexico",
    "San Luis Potosí, Mexico",
    "Culiacán, Mexico",
    "Hermosillo, Mexico",
    "Villahermosa, Mexico",
    "Ciudad Victoria, Mexico",
    "Tlaxcala, Mexico",
    "Xalapa, Mexico",
    "Mérida, Mexico",
    "Zacatecas, Mexico"
]

In [22]:
sc = SparkContext.getOrCreate()
weather_data_list = []

for city in cities:
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        weather_data = {
            'city': data['name'],
            'temperature': data['main']['temp'],
            'description': data['weather'][0]['description'],
            'humidity': data['main']['humidity']
        }
        weather_data_list.append(weather_data)
    else:
        print(f"Error fetching weather for {city}: {response.status_code}")

rdd = sc.parallelize(weather_data_list)
print(rdd.collect())

[{'city': 'Aguascalientes', 'temperature': 14.97, 'description': 'clear sky', 'humidity': 58}, {'city': 'Mexicali', 'temperature': 28.67, 'description': 'clear sky', 'humidity': 10}, {'city': 'La Paz', 'temperature': 10.32, 'description': 'few clouds', 'humidity': 56}, {'city': 'Campeche', 'temperature': 23.88, 'description': 'overcast clouds', 'humidity': 82}, {'city': 'Tuxtla Gutiérrez', 'temperature': 22.45, 'description': 'overcast clouds', 'humidity': 95}, {'city': 'Chihuahua', 'temperature': 19.46, 'description': 'clear sky', 'humidity': 21}, {'city': 'Saltillo', 'temperature': 15.72, 'description': 'clear sky', 'humidity': 59}, {'city': 'Colima', 'temperature': 20.54, 'description': 'few clouds', 'humidity': 94}, {'city': 'Durango', 'temperature': 13.42, 'description': 'clear sky', 'humidity': 42}, {'city': 'Guanajuato City', 'temperature': 18.44, 'description': 'broken clouds', 'humidity': 67}, {'city': 'Chilpancingo', 'temperature': 17.86, 'description': 'moderate rain', 'humi

In [39]:
inicio = time.time()
city_descr_rdd = rdd.map(lambda x: (x['description'], x['city']))
print(city_descr_rdd.collect())
fin = time.time() # Fin de ejecución

print("Tiempo sin cache:", fin - inicio)


[('clear sky', 'Aguascalientes'), ('clear sky', 'Mexicali'), ('few clouds', 'La Paz'), ('overcast clouds', 'Campeche'), ('overcast clouds', 'Tuxtla Gutiérrez'), ('clear sky', 'Chihuahua'), ('clear sky', 'Saltillo'), ('few clouds', 'Colima'), ('clear sky', 'Durango'), ('broken clouds', 'Guanajuato City'), ('moderate rain', 'Chilpancingo'), ('broken clouds', 'Pachuca'), ('clear sky', 'Guadalajara'), ('broken clouds', 'Toluca'), ('broken clouds', 'Morelia'), ('broken clouds', 'Cuernavaca'), ('few clouds', 'Tepic'), ('clear sky', 'Monterrey'), ('broken clouds', 'Oaxaca City'), ('overcast clouds', 'Puebla'), ('scattered clouds', 'Querétaro'), ('light rain', 'Chetumal'), ('clear sky', 'San Luis Potosí'), ('few clouds', 'Culiacán'), ('clear sky', 'Hermosillo'), ('broken clouds', 'Villahermosa'), ('clear sky', 'Ciudad Victoria'), ('overcast clouds', 'Tlaxcala'), ('broken clouds', 'Xalapa'), ('clear sky', 'Mérida'), ('clear sky', 'Zacatecas')]
Tiempo sin cache: 0.47708892822265625


In [33]:
grouped_by_description = city_descr_rdd.groupByKey()
result = grouped_by_description.collect()
for description, cities in result:
    print(f"Description: {description}")
    print(f"Cities: {', '.join(cities)}")

Description: clear sky
Cities: Aguascalientes, Mexicali, Chihuahua, Saltillo, Durango, Guadalajara, Monterrey, San Luis Potosí, Hermosillo, Ciudad Victoria, Mérida, Zacatecas
Description: broken clouds
Cities: Guanajuato City, Pachuca, Toluca, Morelia, Cuernavaca, Oaxaca City, Villahermosa, Xalapa
Description: moderate rain
Cities: Chilpancingo
Description: scattered clouds
Cities: Querétaro
Description: few clouds
Cities: La Paz, Colima, Tepic, Culiacán
Description: overcast clouds
Cities: Campeche, Tuxtla Gutiérrez, Puebla, Tlaxcala
Description: light rain
Cities: Chetumal


In [36]:
city_descr_rdd.cache()

PythonRDD[27] at collect at <ipython-input-30-07d36b3e5fcf>:2

In [41]:
city_descr_rdd.getNumPartitions()

2

In [42]:
city_descr_rdd.take(5)

[('clear sky', 'Aguascalientes'),
 ('clear sky', 'Mexicali'),
 ('few clouds', 'La Paz'),
 ('overcast clouds', 'Campeche'),
 ('overcast clouds', 'Tuxtla Gutiérrez')]